In [3]:
from tensorflow import keras  # tensorflow v1.14.0 was used
import numpy as np            # numpy v1.17.1 was used

np.random.seed(2019)

def make_model():
    inp = keras.layers.Input(shape=(10, 3))
    x = keras.layers.LSTM(10, activation='relu', return_sequences=True)(inp)
    x = keras.layers.LSTM(5, activation='relu', return_sequences=True)(x)
    x = keras.layers.LSTM(1, activation='sigmoid', return_sequences=True)(x)
    out = keras.layers.Flatten()(x)
    return keras.models.Model(inp, out)

def data_gen():
    while True:
        x = np.random.rand(5, 10, 3)  # batch x time x features
        yield x, x[:, :, 0] * x[:, :, 1] < 0.25

def var_importance(model):
    g = data_gen()
    x = np.concatenate([next(g)[0] for _ in range(50)]) # Get a sample of data
    orig_out = model.predict(x)
    for i in range(3):  # iterate over the three features
        new_x = x.copy()
        perturbation = np.random.normal(0.0, 0.2, size=new_x.shape[:2])
        new_x[:, :, i] = new_x[:, :, i] + perturbation
        perturbed_out = model.predict(new_x)
        effect = ((orig_out - perturbed_out) ** 2).mean() ** 0.5
        print(f'Variable {i+1}, perturbation effect: {effect:.4f}')

def main():
    model = make_model()
    model.compile('adam', 'binary_crossentropy')
    print(model.summary())
    model.fit_generator(data_gen(), steps_per_epoch=100, epochs=1)
    var_importance(model)

In [4]:
if __name__ == "__main__":
    main()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10, 3)]           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 10)            560       
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 5)             320       
_________________________________________________________________
lstm_5 (LSTM)                (None, 10, 1)             28        
_________________________________________________________________
flatten_1 (Flatten)          (None, 10)                0         
Total params: 908
Trainable params: 908
Non-trainable params: 0
_________________________________________________________________
None
100/100 [==============================] - 255s 3s/step - loss: 0.8002
Variable 1, perturbation effect: 0.0119
Variable 2,

In [6]:
print(data_gen()

<generator object data_gen at 0x7f84533a7250>
